In [61]:
import numpy as np
import pandas as pd
import magpylib as magpy
import timeit
import numba
from math import sqrt as msqrt
from dipole_sources import *
import matplotlib.pyplot as plt

## MagPy

In [2]:
cube = magpy.magnet.Cuboid(polarization=(1,0,0), dimension=(1,1,1))
sensor = magpy.Sensor()

In [5]:
def calc_all_magpy(
        x_grid: np.ndarray,
        y_grid: np.ndarray,
        location: np.ndarray,
        source_vector: np.ndarray,
        lift_off: float = 0.0) -> np.ndarray:
    
    z_grid = np.ones(x_grid.shape)*lift_off
    dx = np.abs(location[0,0,1]-location[0,0,0])
    
    # build observer array
    obs_arr = np.stack([x_grid.flatten(),y_grid.flatten(),z_grid.flatten()])
    
    B = magpy.getB(
        sources="Cuboid",
        observers=obs_arr,
        dimension=(dx,dx,dx),
        position=location,
        polarization=source_vector)
    
    return B

In [2]:
def B_dipole2(
        observer: np.ndarray,
        polarization: np.ndarray,
        location: np.ndarray) -> np.ndarray:
    """
    Compute the field of a magnetic dipole point source

    """
    mx = polarization[:,0]
    my = polarization[:,1]
    mz = polarization[:,2]
    
    dxyz = observer - location   
    dx = dxyz[:,0]
    dy = dxyz[:,1]
    dz = dxyz[:,2]
    
    
    squared_distance = dx**2 + dy**2 + dz**2
    
    gridsum = mx * dx + my * dy + mz * dz
    print(gridsum)
    aux = gridsum/np.sqrt(squared_distance**5)
    
    
    tmp = 1/ np.sqrt(squared_distance**3)
    
    
    bx = 3.0 * aux * dx - mx * tmp*9.9472e-8
    by = 3.0 * aux * dy - my * tmp*9.9472e-8
    bz = 3.0 * aux * dz - mz * tmp*9.9472e-8
    
    return bx,by,bz

In [3]:
from dipole_sources import B_dipole
B_dipole(np.array([[10,10,1]]),np.array([[0,1,0]]),np.array([[5,5,5]]))

(array([0.00211934]), array([0.00211934]), array([-0.00169548]))

In [8]:
timeit.Timer("magpy.getB(sources='Cuboid',observers=(10,10,1),dimension=(1,1,1),position=(0,0,0),polarization=(0,1,0))",setup='import magpylib as magpy').timeit(number=1000)

0.3332087000599131

In [6]:
timeit.Timer("magpy.core.dipole_Hfield(np.array([[10,10,1]]),np.array([[0,1,0]]))",setup='import magpylib as magpy; import numpy as np').timeit(number=1000)


0.03710509999655187

In [89]:
timeit.Timer("B_dipole(np.array([[10,10,1]]),np.array([[0,1,0]]),np.array([[5,5,5]]))",setup='import magpylib as magpy; import numpy as np; from dipole_sources import B_dipole').timeit(number=1000)

0.005509299924597144

In [87]:
timeit.Timer("magnetic_field(10,10,1,-1,1,-1,1,-1,1,1,0,0)",setup='from choclo.prism import magnetic_field; import numpy as np').timeit(number=1000)

0.0008314000442624092

2222.222222222222

## FMM

In [ ]:
import treecode_helper

def make_Points(self, mag_moms, xm_list, ym_list, zm_list,epsilon=1e-6):
    
    particles = []
    
    # loop over domain and create particles
    # this is slow but we only have to do it once
    for x,i in enumerate(xm_list):
        for y,j in enumerate(ym_list):
            for z,k in enumerate(zm_list):
                if mag_moms[i,j,k]*mag_moms[i,j,k] > epsilon*epsilon:
                    particles.append(Particle(coords=(x,y,z),m=mag_moms[i,j,k]))

In [7]:
dx =np.array([[1,2,3,4]])

In [14]:
np.matmul(dx.reshape(-1,1),np.ones(dx.shape))

array([[1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.],
       [4., 4., 4., 4.]])

In [92]:
dist_crit = 9

# all unique sizes of root cells
cell_lens = np.array([1,2,4])


# get maximum number of cells of each size which would be considered near-field
n_crit = np.ceil(dist_crit/cell_lens).astype('int')+1

dists = np.zeros((len(cell_lens),np.max(n_crit)))

for i,cell_size in enumerate(cell_lens):
    dists[i,:] = cell_size*np.arange(dists.shape[1])
    
   
dists[np.floor(dists/dist_crit) > 1] = 0

# all possible combinations of displacements to near field cells
# Tm=np.matmul(dists.reshape(-1,1)/2,np.ones((1,len(dists))))
# dists = np.unique(Tm+Tm.T)

dists
# # drop out combinations that are much larger than critical distance
# dists = dists[np.floor(dists/dist_crit)>1]
# B_cubes_x = B_cube_dict_choclo(,[1,0,0],np.zeros(len),cell_size)
# B_cubes_y = B_cube_dict_choclo(observers,[0,1,0],np.zeros(len),cell_size)
# B_cubes_z = B_cube_dict_choclo(observers,[0,0,1],np.zeros(len),cell_size)
# lx, ly, lz = np.meshgrid((dists,dists,dists))

# observers = 



array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16.,  0.],
       [ 0.,  4.,  8., 12., 16.,  0.,  0.,  0.,  0.,  0.]])

In [70]:
dists-cell_lens.reshape(-1,1)/2

array([[ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5],
       [ 1. ,  3. ,  5. ,  7. ,  9. , 11. , 13. , 15. , -1. , -1. ],
       [ 2. ,  6. , 10. , 14. , -2. , -2. , -2. , -2. , -2. , -2. ]])

In [2]:
n = 2.
easting = np.arange(-n,n+.1, 1.)
northing = np.arange(-n,n+.1, 1.)
upward = np.arange(-n,n+.1, 1.)

obs= np.array([[0.,0.,0.],[100.,100.,100.]]).T
eastingM, northingM, upwardM = np.meshgrid(easting, northing,upward)

coordstack =np.stack([eastingM.ravel(),northingM.ravel(),upwardM.ravel()],axis=1)
center_ind = np.argmin(np.sum(np.abs(coordstack),axis=1))
# coordstack = coordstack[np.any(np.abs(coordstack) > 0, axis=1),:]


prism_coordinates = np.stack([coordstack[:,0]-0.5,coordstack[:,0]+0.5, coordstack[:,1]-0.5, coordstack[:,1]+0.5,coordstack[:,2]-0.5,coordstack[:,2]+0.5],axis=1)

# J1=build_jacobian(coordinates,np.array([[-.5,.5,-.5,.5,-.5,.5]]))
J1=build_jacobian_choclo(obs,prism_coordinates)
# J1=J1.reshape(int(2*n+1),int(2*n+1),int(2*n+1),3)

array([[[[-2., -2., -2.],
         [-2., -1., -2.],
         [-2.,  0., -2.],
         [-2.,  1., -2.],
         [-2.,  2., -2.]],

        [[-1., -2., -2.],
         [-1., -1., -2.],
         [-1.,  0., -2.],
         [-1.,  1., -2.],
         [-1.,  2., -2.]],

        [[ 0., -2., -2.],
         [ 0., -1., -2.],
         [ 0.,  0., -2.],
         [ 0.,  1., -2.],
         [ 0.,  2., -2.]],

        [[ 1., -2., -2.],
         [ 1., -1., -2.],
         [ 1.,  0., -2.],
         [ 1.,  1., -2.],
         [ 1.,  2., -2.]],

        [[ 2., -2., -2.],
         [ 2., -1., -2.],
         [ 2.,  0., -2.],
         [ 2.,  1., -2.],
         [ 2.,  2., -2.]]],


       [[[-2., -2., -1.],
         [-2., -1., -1.],
         [-2.,  0., -1.],
         [-2.,  1., -1.],
         [-2.,  2., -1.]],

        [[-1., -2., -1.],
         [-1., -1., -1.],
         [-1.,  0., -1.],
         [-1.,  1., -1.],
         [-1.,  2., -1.]],

        [[ 0., -2., -1.],
         [ 0., -1., -1.],
         [ 0.,  0., -1

In [13]:
J1[0,:,:].reshape(5,5,5,3)

array([[[[-1.11022303e-23,  2.40816756e-09,  2.40816756e-09],
         [ 1.23636776e-09,  4.94794535e-09,  2.46916312e-09],
         [ 2.20745479e-09,  6.63782313e-09,  0.00000000e+00],
         [ 1.23636776e-09,  4.94794535e-09, -2.46916312e-09],
         [-1.66533454e-23,  2.40816756e-09, -2.40816756e-09]],

        [[-2.47273552e-09,  2.46916312e-09,  2.46916312e-09],
         [-3.42576428e-09,  6.80730380e-09,  3.38366724e-09],
         [-3.64791364e-09,  1.06794372e-08,  0.00000000e+00],
         [-3.42576428e-09,  6.80730380e-09, -3.38366724e-09],
         [-2.47273552e-09,  2.46916312e-09, -2.46916312e-09]],

        [[-4.41490959e-09,  0.00000000e+00,  5.55111513e-24],
         [-8.90090474e-09, -5.55111513e-24, -1.11022303e-23],
         [-1.23392788e-08,  0.00000000e+00,  0.00000000e+00],
         [-8.90090474e-09,  0.00000000e+00,  0.00000000e+00],
         [-4.41490959e-09,  0.00000000e+00, -1.11022303e-23]],

        [[-2.47273552e-09, -2.46916312e-09, -2.46916312e-09],
  

In [8]:
np.random.seed(24)
M=np.random.rand(int(2*n+1),int(2*n+1),int(2*n+1))*100

In [9]:
# M_rav=np.delete(M.ravel(),center_ind)
M_rav= M.ravel()

In [10]:
[J1[0,:,0]@M_rav,J1[0,:,1]@M_rav,J1[0,:,2]@M_rav]

[np.float64(1.9168143691230713e-06),
 np.float64(6.4529752886723485e-06),
 np.float64(-7.78834573182949e-07)]

In [84]:
[J1[0,:,0]@M_rav,J1[0,:,1]@M_rav,J1[0,:,2]@M_rav]

[np.float64(1.916814369123072e-06),
 np.float64(6.452975288672349e-06),
 np.float64(-7.788345731829485e-07)]

In [148]:
np.nansum(B22[0,:,:],axis=0)

array([ 1.91681437e-06,  6.45297529e-06, -7.78834573e-07])

In [97]:
np.nansum(B22,axis=1)

array([[ 3.20825436e-05, -3.38335671e-06, -3.72824987e-06]])

In [21]:
np.nansum(J1.reshape(-1,3)*M1,axis=0)

array([ 1.91681437e-06,  6.45297529e-06, -7.78834573e-07])

In [29]:
np.nansum(B23.reshape(-1,3)*M1,axis=0)

NameError: name 'B23' is not defined

In [32]:
np.nansum(B22,axis=1)*np.nansum(J1[:,:,:,2]*M)/8.70356566e-06

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [83]:
coordstack

array([[-5., -5., -5.],
       [-5., -5., -4.],
       [-5., -5., -3.],
       ...,
       [ 5.,  5.,  3.],
       [ 5.,  5.,  4.],
       [ 5.,  5.,  5.]])

In [14]:
def B_cube_dict_choclo2(
        observer: np.ndarray,
        polarization: np.ndarray,
        location: np.ndarray,
        dx: np.ndarray,
        ) -> np.ndarray:
    """
    Returns magnetic field components for all combinations of source and observer. Used to assemble 

    Args:
        observer (np.ndarray): _description_
        polarization (np.ndarray): _description_
        location (np.ndarray): _description_
        dx (np.ndarray): _description_

    Returns:
        np.ndarray: _description_
    """
    
    if (observer.shape[0] != 3) and (observer.shape[1] == 3):
        observer = observer.T
        
    if (location.shape[0] != 3) and (location.shape[1] == 3):
        location = location.T
        
    if (polarization.shape[0] != 3) and (polarization.shape[1] == 3):
        polarization = polarization.T
        
    # Unpack coordinates of the observation points
    easting, northing, upward = observer[:]
    
    
    # Initialize a result array full of zeros
    result = np.zeros((easting.shape[0],location.shape[1],3), dtype=np.float64)
    result2 = np.zeros((easting.shape[0],location.shape[1],3), dtype=np.float64)
    
    # Compute the upward component that every prism generate on each
    # observation point
    for i in range(len(easting)):
        for j in range(location.shape[1]):
            result[i,j,:] = choclo.prism.magnetic_field(
                easting[i],
                northing[i],
                upward[i],
                location[0,j]-dx[j],
                location[0,j]+dx[j],
                location[1,j]-dx[j],
                location[1,j]+dx[j],
                location[2,j]-dx[j],
                location[2,j]+dx[j],
                # polarization[0,j],
                # polarization[1,j],
                # polarization[2,j],
                1.,
                0.,
                0.,
                )
            
            result2[i,j,:] = choclo.prism.magnetic_field(
                easting[i],
                northing[i],
                upward[i],
                location[0,j]-dx[j],
                location[0,j]+dx[j],
                location[1,j]-dx[j],
                location[1,j]+dx[j],
                location[2,j]-dx[j],
                location[2,j]+dx[j],
                # polarization[0,j],
                # polarization[1,j],
                # polarization[2,j],
                0.,
                1.,
                0.,
                )
    result = np.nan_to_num(result) 
    result2 = np.nan_to_num(result2)          
    return result,result2

In [191]:
B22x,B22y=B_cube_dict_choclo2(np.array([[0.,0.,0.],[100.,100.,100.]]),np.stack([M_rav,np.zeros(M_rav.shape),np.zeros(M_rav.shape)]),coordstack,0.5*np.ones(M_rav.shape[0]))

In [210]:
from scipy.spatial.transform import Rotation as R
Ry=R.from_euler('xyz', [0,0,-90], degrees=True)
Ry.as_matrix()

array([[ 0.,  1.,  0.],
       [-1.,  0., -0.],
       [-0.,  0.,  1.]])

In [208]:
B22y[0,:,:]

array([[ 2.40816756e-09, -1.11022303e-23,  2.40816756e-09],
       [ 4.94794535e-09,  1.23636776e-09,  2.46916312e-09],
       [ 6.63782313e-09,  2.20745479e-09,  0.00000000e+00],
       [ 4.94794535e-09,  1.23636776e-09, -2.46916312e-09],
       [ 2.40816756e-09, -1.66533454e-23, -2.40816756e-09],
       [ 2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
       [ 6.80730380e-09,  6.85152857e-09,  6.80730380e-09],
       [ 1.06794372e-08,  1.25488184e-08,  0.00000000e+00],
       [ 6.80730380e-09,  6.85152857e-09, -6.80730380e-09],
       [ 2.46916312e-09,  1.23636776e-09, -4.94794535e-09],
       [ 0.00000000e+00,  2.20745479e-09,  6.63782313e-09],
       [-5.55111513e-24,  1.25488184e-08,  1.06794372e-08],
       [ 0.00000000e+00,  2.46785575e-08,  0.00000000e+00],
       [ 0.00000000e+00,  1.25488184e-08, -1.06794372e-08],
       [ 0.00000000e+00,  2.20745479e-09, -6.63782313e-09],
       [-2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
       [-6.80730380e-09,  6.85152857e-09

In [212]:
tmp1=Ry.as_matrix()@B22x[0,:,:].T
tmp1.T

array([[ 2.40816756e-09,  1.11022303e-23,  2.40816756e-09],
       [ 4.94794535e-09, -1.23636776e-09,  2.46916312e-09],
       [ 6.63782313e-09, -2.20745479e-09,  0.00000000e+00],
       [ 4.94794535e-09, -1.23636776e-09, -2.46916312e-09],
       [ 2.40816756e-09,  1.66533454e-23, -2.40816756e-09],
       [ 2.46916312e-09,  2.47273552e-09,  2.46916312e-09],
       [ 6.80730380e-09,  3.42576428e-09,  3.38366724e-09],
       [ 1.06794372e-08,  3.64791364e-09,  0.00000000e+00],
       [ 6.80730380e-09,  3.42576428e-09, -3.38366724e-09],
       [ 2.46916312e-09,  2.47273552e-09, -2.46916312e-09],
       [ 0.00000000e+00,  4.41490959e-09,  5.55111513e-24],
       [-5.55111513e-24,  8.90090474e-09, -1.11022303e-23],
       [ 0.00000000e+00,  1.23392788e-08,  0.00000000e+00],
       [ 0.00000000e+00,  8.90090474e-09,  0.00000000e+00],
       [ 0.00000000e+00,  4.41490959e-09, -1.11022303e-23],
       [-2.46916312e-09,  2.47273552e-09, -2.46916312e-09],
       [-6.80730380e-09,  3.42576428e-09

In [200]:
Ry.as_matrix().shape

(3, 3)

In [26]:
BxM=B22x[0,:,:].reshape(5,5,5,3)
BxM

array([[[[-1.11022303e-23,  2.40816756e-09,  2.40816756e-09],
         [ 1.23636776e-09,  4.94794535e-09,  2.46916312e-09],
         [ 2.20745479e-09,  6.63782313e-09,  0.00000000e+00],
         [ 1.23636776e-09,  4.94794535e-09, -2.46916312e-09],
         [-1.66533454e-23,  2.40816756e-09, -2.40816756e-09]],

        [[-2.47273552e-09,  2.46916312e-09,  2.46916312e-09],
         [-3.42576428e-09,  6.80730380e-09,  3.38366724e-09],
         [-3.64791364e-09,  1.06794372e-08,  0.00000000e+00],
         [-3.42576428e-09,  6.80730380e-09, -3.38366724e-09],
         [-2.47273552e-09,  2.46916312e-09, -2.46916312e-09]],

        [[-4.41490959e-09,  0.00000000e+00,  5.55111513e-24],
         [-8.90090474e-09, -5.55111513e-24, -1.11022303e-23],
         [-1.23392788e-08,  0.00000000e+00,  0.00000000e+00],
         [-8.90090474e-09,  0.00000000e+00,  0.00000000e+00],
         [-4.41490959e-09,  0.00000000e+00, -1.11022303e-23]],

        [[-2.47273552e-09, -2.46916312e-09, -2.46916312e-09],
  

array([[ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.]])

array([[ 2.40816756e-09,  1.11022303e-23,  2.40816756e-09],
       [ 4.94794535e-09, -1.23636776e-09,  2.46916312e-09],
       [ 6.63782313e-09, -2.20745479e-09,  0.00000000e+00],
       [ 4.94794535e-09, -1.23636776e-09, -2.46916312e-09],
       [ 2.40816756e-09,  1.66533454e-23, -2.40816756e-09],
       [ 2.46916312e-09,  2.47273552e-09,  2.46916312e-09],
       [ 6.80730380e-09,  3.42576428e-09,  3.38366724e-09],
       [ 1.06794372e-08,  3.64791364e-09,  0.00000000e+00],
       [ 6.80730380e-09,  3.42576428e-09, -3.38366724e-09],
       [ 2.46916312e-09,  2.47273552e-09, -2.46916312e-09],
       [ 0.00000000e+00,  4.41490959e-09,  5.55111513e-24],
       [-5.55111513e-24,  8.90090474e-09, -1.11022303e-23],
       [ 0.00000000e+00,  1.23392788e-08,  0.00000000e+00],
       [ 0.00000000e+00,  8.90090474e-09,  0.00000000e+00],
       [ 0.00000000e+00,  4.41490959e-09, -1.11022303e-23],
       [-2.46916312e-09,  2.47273552e-09, -2.46916312e-09],
       [-6.80730380e-09,  3.42576428e-09

In [ ]:
pts_mat=np.stack([eastingM, northingM, upwardM]).T

In [156]:
np.all(np.abs(np.stack([-northingM, eastingM, upwardM]).T-pts_mat@Ry.as_matrix())<1e15)

np.True_

In [165]:
pts_mat

array([[[[-2., -2., -2.],
         [-2., -1., -2.],
         [-2.,  0., -2.],
         [-2.,  1., -2.],
         [-2.,  2., -2.]],

        [[-1., -2., -2.],
         [-1., -1., -2.],
         [-1.,  0., -2.],
         [-1.,  1., -2.],
         [-1.,  2., -2.]],

        [[ 0., -2., -2.],
         [ 0., -1., -2.],
         [ 0.,  0., -2.],
         [ 0.,  1., -2.],
         [ 0.,  2., -2.]],

        [[ 1., -2., -2.],
         [ 1., -1., -2.],
         [ 1.,  0., -2.],
         [ 1.,  1., -2.],
         [ 1.,  2., -2.]],

        [[ 2., -2., -2.],
         [ 2., -1., -2.],
         [ 2.,  0., -2.],
         [ 2.,  1., -2.],
         [ 2.,  2., -2.]]],


       [[[-2., -2., -1.],
         [-2., -1., -1.],
         [-2.,  0., -1.],
         [-2.,  1., -1.],
         [-2.,  2., -1.]],

        [[-1., -2., -1.],
         [-1., -1., -1.],
         [-1.,  0., -1.],
         [-1.,  1., -1.],
         [-1.,  2., -1.]],

        [[ 0., -2., -1.],
         [ 0., -1., -1.],
         [ 0.,  0., -1

In [174]:
tmp1=np.argsort(eastingM.ravel())

In [142]:
ByM

array([[[[ 2.40816756e-09, -1.11022303e-23,  2.40816756e-09],
         [ 4.94794535e-09,  1.23636776e-09,  2.46916312e-09],
         [ 6.63782313e-09,  2.20745479e-09,  0.00000000e+00],
         [ 4.94794535e-09,  1.23636776e-09, -2.46916312e-09],
         [ 2.40816756e-09, -1.66533454e-23, -2.40816756e-09]],

        [[ 2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
         [ 6.80730380e-09,  6.85152857e-09,  6.80730380e-09],
         [ 1.06794372e-08,  1.25488184e-08,  0.00000000e+00],
         [ 6.80730380e-09,  6.85152857e-09, -6.80730380e-09],
         [ 2.46916312e-09,  1.23636776e-09, -4.94794535e-09]],

        [[ 0.00000000e+00,  2.20745479e-09,  6.63782313e-09],
         [-5.55111513e-24,  1.25488184e-08,  1.06794372e-08],
         [ 0.00000000e+00,  2.46785575e-08,  0.00000000e+00],
         [ 0.00000000e+00,  1.25488184e-08, -1.06794372e-08],
         [ 0.00000000e+00,  2.20745479e-09, -6.63782313e-09]],

        [[-2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
  

In [187]:
test1=B22x[0,:,:]@np.array([[ 0., 1.,  0.],[ 1.,  0.,  0.],[ 0.,  0.,  1.]])
test1[tmp1]

array([[ 2.40816756e-09, -1.11022303e-23,  2.40816756e-09],
       [ 4.94794535e-09,  1.23636776e-09,  2.46916312e-09],
       [ 6.63782313e-09,  2.20745479e-09,  0.00000000e+00],
       [ 4.94794535e-09,  1.23636776e-09, -2.46916312e-09],
       [ 2.40816756e-09, -1.66533454e-23, -2.40816756e-09],
       [ 6.80730380e-09,  6.85152857e-09,  6.80730380e-09],
       [ 1.06794372e-08,  1.25488184e-08,  0.00000000e+00],
       [ 6.80730380e-09,  6.85152857e-09, -6.80730380e-09],
       [ 2.46916312e-09,  1.23636776e-09, -4.94794535e-09],
       [ 2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
       [ 0.00000000e+00,  1.25488184e-08, -1.06794372e-08],
       [ 0.00000000e+00,  1.25488184e-08,  1.06794372e-08],
       [ 0.00000000e+00,  2.20745479e-09, -6.63782313e-09],
       [ 0.00000000e+00,  2.20745479e-09,  6.63782313e-09],
       [ 0.00000000e+00,  2.46785575e-08,  0.00000000e+00],
       [-6.80730380e-09,  6.85152857e-09,  6.80730380e-09],
       [-4.94794535e-09,  1.23636776e-09

In [190]:
B22y[0,:,:]

array([[ 2.40816756e-09, -1.11022303e-23,  2.40816756e-09],
       [ 4.94794535e-09,  1.23636776e-09,  2.46916312e-09],
       [ 6.63782313e-09,  2.20745479e-09,  0.00000000e+00],
       [ 4.94794535e-09,  1.23636776e-09, -2.46916312e-09],
       [ 2.40816756e-09, -1.66533454e-23, -2.40816756e-09],
       [ 2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
       [ 6.80730380e-09,  6.85152857e-09,  6.80730380e-09],
       [ 1.06794372e-08,  1.25488184e-08,  0.00000000e+00],
       [ 6.80730380e-09,  6.85152857e-09, -6.80730380e-09],
       [ 2.46916312e-09,  1.23636776e-09, -4.94794535e-09],
       [ 0.00000000e+00,  2.20745479e-09,  6.63782313e-09],
       [-5.55111513e-24,  1.25488184e-08,  1.06794372e-08],
       [ 0.00000000e+00,  2.46785575e-08,  0.00000000e+00],
       [ 0.00000000e+00,  1.25488184e-08, -1.06794372e-08],
       [ 0.00000000e+00,  2.20745479e-09, -6.63782313e-09],
       [-2.46916312e-09,  1.23636776e-09,  4.94794535e-09],
       [-6.80730380e-09,  6.85152857e-09

In [114]:
v1=B22x[0,:,:]@np.array([[ 0.,  1.,  0.],[1,  0., 0.],[0.,  0.,  1.]])


In [115]:
np.all(v1[:,0]==B22y[0,:,0])

np.True_

In [116]:
v1[:]-B22y[0,:,:]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.11669383e-23,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  5.58346913e-24,  5.37667398e-24],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.70910328e-09, -2.47878224e-09],
       [ 0.00000000e+00, -1.02772929e-08, -3.42363656e-09],
       [ 0.00000000e+00, -1.61967320e-08,  0.00000000e+00],
       [ 0.00000000e+00, -1.02772929e-08,  3.42363656e-09],
       [ 0.00000000e+00, -3.70910328e-09,  2.47878224e-09],
       [ 0.00000000e+00, -6.62236438e-09, -6.63782313e-09],
       [ 0.00000000e+00, -2.14497231e-08, -1.06794372e-08],
       [ 0.00000000e+00, -3.70178363e-08,  0.00000000e+00],
       [ 0.00000000e+00, -2.14497231e-08,  1.06794372e-08],
       [ 0.00000000e+00, -6.62236438e-09,  6.63782313e-09],
       [ 0.00000000e+00, -3.70910328e-09, -7.41710847e-09],
       [ 0.00000000e+00, -1.02772929e-08

In [81]:
B22y[0,:,:].shape

(125, 3)

In [ ]:
B23=magpy.getB(
        sources="Cuboid",
        observers=np.array([[0,0,0]]),
        dimension=(1.,1.,1.),
        position=np.stack(coordstack,axis=1),
        polarization=M.ravel().reshape(-1,1)@np.array([[1,0,0]]))

In [95]:
B22

array([[[ 6.57758598e-07,  6.53512944e-07, -6.53512944e-07],
        [ 4.79272679e-07,  4.76179104e-07,  4.76179104e-07],
        [ 6.85061932e-07, -6.80640042e-07, -6.80640042e-07],
        [ 2.72084115e-08,  5.43382059e-08, -1.08888097e-07],
        [ 8.91035000e-07,  0.00000000e+00,  0.00000000e+00],
        [ 8.88178420e-22, -1.45131443e-06, -1.45131443e-06],
        [ 1.23198573e-07,  2.46041172e-07,  4.93040847e-07],
        [ 3.96978077e-07, -3.37840768e-07,  0.00000000e+00],
        [-4.71732535e-07,  0.00000000e+00,  0.00000000e+00],
        [ 2.63085001e-07,  2.61386858e-07,  2.61386858e-07],
        [ 7.07531829e-08, -2.12755032e-07,  0.00000000e+00],
        [-1.25525057e-07,  2.49429654e-07,  1.23982560e-07],
        [ 8.90528858e-07,  7.57867928e-07,  0.00000000e+00],
        [ 1.98702372e-07,  5.97498625e-07,  0.00000000e+00],
        [ 6.60363109e-08,  2.64277401e-07,  1.31881814e-07],
        [ 8.88178420e-23,  4.85105602e-07, -4.85105602e-07],
        [ 8.43037854e-07

In [91]:
J1[:,:,:,2]*M

array([[[ 1.47808191e-08,  1.18603569e-08,  1.68279306e-08, ...,
         -2.29806768e-09, -6.51045247e-09, -4.93484600e-09],
        [ 6.21263027e-09,  1.38870516e-08,  1.83311914e-08, ...,
         -1.81948528e-08, -1.64922107e-08, -5.18850036e-09],
        [ 1.06226909e-08,  1.38529063e-08,  2.16091956e-08, ...,
         -1.08228148e-08, -1.64388334e-08, -1.42083637e-08],
        ...,
        [-1.49206102e-09, -4.06048373e-09, -1.05093039e-08, ...,
          1.34492451e-08,  1.11186101e-08,  3.53079693e-09],
        [-2.30798660e-09, -1.50516134e-08, -9.66479802e-09, ...,
          5.06739473e-09,  5.02448076e-09,  3.89868643e-10],
        [-1.43556901e-08, -6.65082439e-09, -1.09107829e-08, ...,
          1.37435743e-08,  4.90583476e-09,  1.00244687e-08]],

       [[ 3.25767911e-09,  1.29223349e-08,  1.73652897e-08, ...,
         -1.30430828e-09, -2.23171700e-08, -9.26280961e-09],
        [ 1.55563657e-08,  2.47019715e-08,  5.89385402e-09, ...,
         -2.30689434e-08, -8.60805412e

In [17]:
prisms=np.stack((eastingM.ravel()-0.5,eastingM.ravel()+0.5,northingM.ravel()-0.5,northingM.ravel()+0.5,upwardM.ravel()-0.5,upwardM.ravel()+0.5)).T

In [31]:
J1=build_jacobian(coordinates,np.array([[-.5,.5,-.5,.5,-.5,.5]]))

In [32]:
J1.shape[0]*64*1.25e-10

0.00023832800000000003

In [40]:
31*31*31*3

89373